In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm

from scipy.interpolate import interpn
from tqdm import tqdm

Read **STEFAN HEIGLS** data. Data is currently (until 01.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/4abe80f1c550806021f85af8c57c886e).

In [ ]:
f = np.load('turbulentbox.npy')
data = f.copy()
del f

Read **THOMAS PFEILS** data. Data is currently (until 02.02.2022) available [here](https://gigamove.rwth-aachen.de/de/download/ebdbc12c85438bd224a5015206840893).

In [25]:
Slider?

In [2]:
with np.load('pluto_data.npz') as f:
    data = f['rho']

# Visualization

## Pyvista

In [ ]:
import pyvista

In [ ]:
grid = pyvista.UniformGrid()
grid.dimensions = np.array(data.shape) + 1
grid.spacing = (1, 1, 1)
grid.cell_data["density"] = np.log10(data.flatten(order="F"))

vmax = grid['density'].max()
grid['opac'] = Normalize(vmin=vmax - 1.5, vmax=vmax)(grid['density'])

In [ ]:
pyvista.close_all()

In [ ]:
p = pyvista.Plotter()
p.set_background('white')
p.add_volume(grid, scalars='opac', cmap="viridis", clim=[0, 1], opacity='linear', shade=False, opacity_unit_distance=.010)
p.show()

## Volumerender

Define the transfer function: it assigns color values and alpha values to each density

In [3]:
def transferFunction(x, x0, A, sigma, colors):
    
    extra_dims = tuple(np.arange(x.ndim))
    
    x0     = np.expand_dims(x0, axis=extra_dims)
    A      = np.expand_dims(A, axis=extra_dims)
    sigma  = np.expand_dims(sigma, axis=extra_dims)
    colors = np.expand_dims(colors, axis=extra_dims)

    assert x0.shape == A.shape == sigma.shape, 'shapes of x0, A, and sigma must match'
    
    vals = colors[..., :, :]  * A[..., :, None] *  np.exp(-(x[..., None, None] - x0[..., :, None] )**2 / (2 * sigma[..., :, None]**2))

    return vals.sum(-2).T

Define function that roates and ray-traces image

In [5]:
def makeslice(phi, theta, N=180):

    # Camera Grid / Query Points -- rotate camera view
    c = np.linspace(-N / 2, N / 2, N)
    qx, qy, qz = np.meshgrid(c, c, c)
    # qxR = qx
    # qyR = qy * np.cos(angle) - qz * np.sin(angle)
    # qzR = qy * np.sin(angle) + qz * np.cos(angle)
    
    qxR = qx * np.cos(phi)   - qy * np.sin(phi) * np.cos(theta) + qz * np.sin(phi) * np.sin(theta)
    qyR = qx * np.sin(phi)   + qy * np.cos(phi) * np.cos(theta) - qz * np.sin(theta) * np.cos(phi)
    qzR = qy * np.sin(theta) + qz * np.cos(theta)
    
    qi = np.array([qxR.ravel(), qyR.ravel(), qzR.ravel()]).T

    # Interpolate onto Camera Grid
    camera_grid = interpn(points, datacube, qi, method='linear', bounds_error=False, fill_value=0.0).reshape((N, N, N))

    # Do Volume Rendering
    image = np.zeros((camera_grid.shape[1], camera_grid.shape[2], 3))

    for dataslice in camera_grid:
        r, g, b, a = transferFunction(dataslice, x0, A, sigma, colors)
        image[:, :, 0] = a * r + (1 - a) * image[:, :, 0]
        image[:, :, 1] = a * g + (1 - a) * image[:, :, 1]
        image[:, :, 2] = a * b + (1 - a) * image[:, :, 2]

    image = np.clip(image, 0.0, 1.0)
    return image

In [7]:
N = 100
phi = 0.0
theta = np.pi / 4

# Camera Grid / Query Points -- rotate camera view
c = np.linspace(-N / 2, N / 2, N)
qx, qy, qz = np.meshgrid(c, c, c)
# qxR = qx
# qyR = qy * np.cos(angle) - qz * np.sin(angle)
# qzR = qy * np.sin(angle) + qz * np.cos(angle)

qxR = qx * np.cos(phi)   - qy * np.sin(phi) * np.cos(theta) + qz * np.sin(phi) * np.sin(theta)
qyR = qx * np.sin(phi)   + qy * np.cos(phi) * np.cos(theta) - qz * np.sin(theta) * np.cos(phi)
qzR = qy * np.sin(theta) + qz * np.cos(theta)

qi = np.array([qxR.ravel(), qyR.ravel(), qzR.ravel()]).T

In [10]:
camera_grid = interpn((x, y, z), datacube, qi, method='linear', bounds_error=False, fill_value=0.0).reshape((N, N, N))

In [111]:
from scipy.interpolate import LinearNDInterpolator

In [11]:
from scipy.interpolate import RegularGridInterpolator

In [12]:
p = np.array(np.meshgrid(x,y,z, indexing='ij'))
print(p.shape)
p = p.reshape([3, -1])

(3, 500, 515, 72)


In [16]:
interp = RegularGridInterpolator((x, y, z), datacube, bounds_error=False, fill_value=0.0)

camera_grid2 = interp(qi).reshape((N, N, N))

Normalize data to 0...1 and define grid points

In [9]:
vmax = data.max()
datacube = LogNorm(vmin=vmax * 1e-4, vmax=vmax, clip=True)(data)
# Datacube Grid
Nx, Ny, Nz = datacube.shape
x = np.linspace(-Nx / 2, Nx / 2, Nx)
y = np.linspace(-Ny / 2, Ny / 2, Ny)
z = np.linspace(-Nz / 2, Nz / 2, Nz)
points = (x, y, z)

Show some statistics of the data

In [ ]:
from scipy.integrate import cumtrapz

f, axs = plt.subplots(1, 2, figsize=(10, 4))

ax = axs[0]
ax.plot(z, cumtrapz(datacube, x=z, axis=-1, initial=0).mean(0).mean(0), 'r', lw=3)
for i in range(50):
    ix = np.random.choice(np.arange(len(x)))
    iy = np.random.choice(np.arange(len(y)))
    ax.plot(z, cumtrapz(datacube[ix, iy, :], x=z, initial=0), 'k', alpha=0.1)
ax.set_xlabel('z')

ax = axs[1]
counts, edges, bars = ax.hist(datacube.ravel(), 50)
ax.set_yscale('log')

Set the parameters of the transfer function

In [ ]:
x0    = np.array([0.2, 0.4, 0.9])
A     = np.array([0.1, 0.1, 0.1])
sigma = [0.02, 0.02, 0.02]
colors = np.array([
    [1.0, 0.0, 0.0, 1e-2],
    [0.0, 1.0, 0.0, 5e-2],
    [0.0, 0.0, 1.0, 1e-1],
])

In [ ]:
f, ax = plt.subplots()
counts, edges, bars = ax.hist(datacube.ravel(), 50, density=True)
centers = 0.5 * (edges[1:] + edges[:-1])

r, g, b, a = transferFunction(centers, x0, A, sigma, colors)
_r, _g, _b, _a = volumerender.transferFunction(centers)

ax.plot(centers, r, 'r', label='r')
ax.plot(centers, g, 'g', label='g')
ax.plot(centers, b, 'b', label='b')

ax.plot(centers, _r, '--r', label='_r')
ax.plot(centers, _g, '--g', label='_g')
ax.plot(centers, _b, '--b', label='_b')

ax.plot(centers, a / colors[:,-1].max(), 'k--', label='a');
ax.semilogy()
ax.set_ylim(1e-3, 1e1)

In [ ]:
plt.imshow

In [ ]:
# make the plot
f,ax = plt.subplots(figsize=(4, 4), dpi=150)
image = makeslice(0, np.pi/4, N=300)
# pimg = ax.imshow(255 * Normalize()(image, clip=True))
pimg = ax.imshow(255 * image)
ax.axis('off')
print(image.max())

Make a movie

In [ ]:
# prepare output folder
frames_path = Path('frames')
if not frames_path.is_dir():
    frames_path.mkdir()
    
# make the plot
f,ax = plt.subplots(figsize=(4, 4), dpi=150)
pimg = ax.imshow(255 * image)
ax.axis('off')
    
# make the loop    
n_angles = 25
phi = np.deg2rad(np.linspace(45, 20, n_angles))
theta = np.deg2rad(np.linspace(45, 20, n_angles))
    
for i, (_phi, _theta) in enumerate(tqdm(zip(phi, theta), total=n_angles)):
    image = makeslice(_phi, _theta, N=300)
    pimg.set_array(255 * image)
    f.savefig(frames_path / f'frame_{i:03d}.jpg', bbox_inches='tight', dpi=200)

In [ ]:
!ffmpeg -y -i {frames_path}/frame_%03d.jpg -c:v libx264 -crf 15 -maxrate 400k -pix_fmt yuv420p -r 20 -bufsize 1835k movie_01.mp4

In [103]:
from IPython.display import HTML
HTML(f"""
<video width="500" controls>
  <source src="movie_01.mp4" type="video/mp4">
</video>
""")

In [ ]:
!open frames

## YT

In [ ]:
import yt

In [ ]:
plt.style.use({'figure.dpi':150})

In [ ]:
with np.load('pluto_data.npz') as f:
    data = f['rho']

In [ ]:
data_dict = dict(density = (data, "cm**-3"))
bbox = np.array([[-1, 1], [-1, 1], [-1, 1]])
ds = yt.load_uniform_grid(data_dict, data.shape, bbox=bbox)

In [ ]:
slc = yt.SlicePlot(ds, "z", ("gas", "density"))
slc.set_cmap(("gas", "density"), "Blues")
slc.annotate_grids(cmap=None)
slc.show()

In [ ]:
sc = yt.create_scene(ds)

# Get a reference to the VolumeSource associated with this scene
# It is the first source associated with the scene, so we can refer to it
# using index 0.
source = sc[0]

# Set the bounds of the transfer function
source.tfh.set_bounds((1e-23, 1e-20))

# set that the transfer function should be evaluated in log space
source.tfh.set_log(True)

# Make underdense regions appear opaque
source.tfh.grey_opacity = False

# Plot the transfer function, along with the CDF of the density field to
# see how the transfer function corresponds to structure in the CDF
source.tfh.plot("transfer_function.png", profile_field=("gas", "density"))

# save the image, flooring especially bright pixels for better contrast
sc.save("rendering.pdf", sigma_clip=6.0)

In [ ]:
!open rendering.pdf